In [1]:
import  pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 18:29:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [5]:
#!gzip -dc fhvhv_tripdata_2021-01.csv.gz

In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [7]:
df.schema


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [8]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [9]:
!pip3 install pandas
import pandas as pd


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:
df_pandas = pd.read_csv('head.csv')

In [11]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [13]:
from pyspark.sql import types

In [14]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [16]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [17]:
df = df.repartition(24)

In [18]:
df.write.parquet('fhvhv/2021/01/')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/arundharavath/Downloads/Projects/DE_Zcp/batch-wk5/fhvhv/2021/01 already exists. Set mode as "overwrite" to overwrite the existing path.

In [19]:
df = spark.read.parquet('fhvhv/2021/01/')

In [20]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [21]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-04 19:26:08|2021-01-04 19:32:11|          35|          72|   NULL|
|           HV0003|              B02872|2021-01-02 17:23:32|2021-01-02 17:45:22|          73|         129|   NULL|
|           HV0005|              B02510|2021-01-03 01:26:11|2021-01-03 01:31:32|         116|         166|   NULL|
|           HV0005|              B02510|2021-01-03 15:37:11|2021-01-03 15:49:52|         244|         119|   NULL|
|           HV0005|              B02510|2021-01-01 16:42:14|2021-01-01 16:49:00|         181|         181|   NULL|
|           HV0003|              B02875|2021-01-03 18:48:12|2021-01-03 19:04:27|

In [22]:
from pyspark.sql import functions as F


In [23]:
def rdm(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
rdm('B02884')

's/b44'

In [26]:
rdm_udf = F.udf(rdm, returnType=types.StringType())

In [28]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', rdm_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-04|  2021-01-04|          35|          72|
|  e/b38| 2021-01-02|  2021-01-02|          73|         129|
|  e/9ce| 2021-01-03|  2021-01-03|         116|         166|
|  e/9ce| 2021-01-03|  2021-01-03|         244|         119|
|  e/9ce| 2021-01-01|  2021-01-01|         181|         181|
|  e/b3b| 2021-01-03|  2021-01-03|         142|         146|
|  a/a7a| 2021-01-02|  2021-01-02|         250|         213|
|  e/b3c| 2021-01-02|  2021-01-02|         255|         237|
|  e/b3c| 2021-01-02|  2021-01-02|         205|         205|
|  e/b35| 2021-01-05|  2021-01-05|          13|         146|
|  e/b38| 2021-01-04|  2021-01-04|         236|         170|
|  e/b38| 2021-01-01|  2021-01-01|          48|          68|
|  e/b14| 2021-01-04|  2021-01-04|         237|         236|
|  e/b3f| 2021-01-01|  2